## Setup Device & Environment

In [1]:
from datasets import load_dataset
import numpy as np

import torch
from transformers import AutoImageProcessor
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda')

device

device(type='cuda')

## Data Preparation

### Load Dataset

In [3]:
raw_dataset = load_dataset("imagefolder", data_dir="./datasets/chest_xray")

Resolving data files: 100%|██████████| 624/624 [00:00<00:00, 208163.98it/s]


In [4]:
print(raw_dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5216
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 624
    })
})


### Setup Labels

In [5]:
labels = labels = raw_dataset["train"].features["label"].names
print(labels)

['NORMAL', 'PNEUMONIA']


In [6]:
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
  label2id[i] = label
  id2label[label] = i

In [7]:
print(label2id)
print(id2label)

{0: 'NORMAL', 1: 'PNEUMONIA'}
{'NORMAL': 0, 'PNEUMONIA': 1}


### Transforming Data

In [8]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [9]:
size = (image_processor.size["height"], image_processor.size["width"])
resizer = RandomResizedCrop(size)
normalize = Normalize(image_processor.image_mean, image_processor.image_std)

In [10]:
_transforms = Compose([resizer, ToTensor(), normalize])

In [11]:
def transforms(examples):
  examples["image"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
  # del examples["image"]
  return examples

In [12]:
dataset = raw_dataset.with_transform(transforms)

In [13]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5216
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 624
    })
})


### Preparing metrics for the model

In [14]:
accuracy = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
  predictions = np.argmax(eval_pred.predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

## Loading Best Model

In [16]:
def get_n_params(model):
  pp=0
  for p in list(model.parameters()):
    nn=1
    for s in list(p.size()):
      nn = nn*s
    pp += nn
  return pp

In [17]:
from own_model import MyCompositeModel

PATH = './model_20231222_094427_2'
model = MyCompositeModel()

model.load_state_dict(torch.load(PATH))
model.eval()

get_n_params(model.my_new_layers)

82770

## Training Setup

### Data Loaders

In [18]:
training_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=16, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=16, shuffle=True)

### Loss Function

In [19]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [20]:
# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(model.my_new_layers.parameters(), lr=0.001, momentum=0.9)

In [21]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=2).to(device)

## Training Model

In [22]:
from training_own import train_one_epoch, train_many_epochs

In [25]:
model = model.to('cpu')
model.eval()

N_EXAMPLES = 20

cum_error = 0

testing_fragment = dataset['test'].shuffle(seed=1)[:N_EXAMPLES]
inputs = testing_fragment['image']
labels_true = testing_fragment['label']

for i in range(len(inputs)):
  input = inputs[i]
  label_true = labels_true[i]
  logits = model(input[None, ...]).detach().numpy()
  label_pred = np.argmax(logits)
  
  cum_error += abs(label_pred - label_true)
  print('index {}: true/predicted: {}/{}'.format(i, label_true, label_pred))
  
error = cum_error / N_EXAMPLES

accuracy = 1 - error

print('testing accuracy: {}'.format(accuracy))

index 0: true/predicted: 0/0
index 1: true/predicted: 1/1
index 2: true/predicted: 1/1
index 3: true/predicted: 1/1
index 4: true/predicted: 1/1
index 5: true/predicted: 0/0
index 6: true/predicted: 0/0
index 7: true/predicted: 0/1
index 8: true/predicted: 0/0
index 9: true/predicted: 1/1
index 10: true/predicted: 0/0
index 11: true/predicted: 1/1
index 12: true/predicted: 1/1
index 13: true/predicted: 1/0
index 14: true/predicted: 0/0
index 15: true/predicted: 1/1
index 16: true/predicted: 1/1
index 17: true/predicted: 1/1
index 18: true/predicted: 1/1
index 19: true/predicted: 0/0
testing accuracy: 0.9


In [ ]:
model = model.to(device)
train_one_epoch(
  model=model,
  training_loader=training_loader,
  optimizer=optimizer,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  cuda_device=device,
  epoch_index=10,
  logging_frequency=25
)

###